In [17]:
# Dependencies and Setup
import pandas as pd
import numpy as np

In [19]:
file_to_load = "Resources/purchase_data.csv"
purchase_data = pd.read_csv(file_to_load)
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [20]:
#Display the total number of players
player_count = purchase_data.SN.nunique()
player_count

576

In [ ]:
#Calculations to obtain number of unique items, average price, etc.
item_num = purchase_data["Item ID"].nunique()
avg_price = purchase_data["Price"].mean()
total_purchases = purchase_data["Purchase ID"].count()
net = purchase_data["Price"].sum()
summary = pd.DataFrame({"Number of Items":[item_num],
                       "Average Price":[avg_price],
                       "Total Purchases":[total_purchases],
                       "Revenue":[net]})
summary

In [21]:
#Gender Demographics
males = purchase_data[purchase_data['Gender'].isin(['Male'])]
females = purchase_data[purchase_data['Gender'].isin(['Female'])]
others = purchase_data[purchase_data['Gender'].isin(["Other / Non-Disclosed"])]

# Returns the number of unique occurrences
male = males.SN.nunique()
female = females.SN.nunique()
other = others.SN.nunique()

gender_demographics = pd.DataFrame({"Gender":["Male", "Female", "Other / Non-Disclosed"],
                   "Purchase Count":[male, female, other],
                   "Percentage":[round(male/player_count*100, 2), round(female/player_count*100,2), round(other/player_count*100,2)]
                  })

gender_demographics = gender_demographics.style.format({
    'var1': '{:,.2f}'.format,
    'var2': '{:,.2f}'.format,
    'var3': '{:,.2%}'.format,
})

gender_demographics

,Gender,Purchase Count,Percentage
0,Male,484,84.030000
1,Female,81,14.060000
2,Other / Non-Disclosed,11,1.910000


In [ ]:
#Purchasing Analysis (Gender)
genders = ["Male","Female","Other / Non-Disclosed"]
total_spent_per_gender = [males["Price"].sum(), females["Price"].sum(), others["Price"].sum()]
avg_spent_per_gender = [males["Price"].mean(), females["Price"].mean(), others["Price"].mean()]
avg_spent_per_person_per_gender = [males["Price"].sum()/male, females["Price"].sum()/female, others["Price"].sum()/other]

purchase_summary_by_gender = pd.DataFrame({"Gender":genders,
    "Purchase Count":[males["Purchase ID"].count(), females["Purchase ID"].count(), others["Purchase ID"].count()],
                               "Average Spent":avg_spent_per_gender,
                               "Total Spent":total_spent_per_gender,
                               "Average Spent Per Person":avg_spent_per_person_per_gender})
purchase_summary_by_gender

In [22]:
#Age Demographics
bins = [0,9,14,19,24,29,34,39,100]
labels = ["<10", "10-14","15-19","20-24","25-29","30-34","35-39","40+"]

In [23]:
purchase_data_copy = purchase_data.copy(deep="True")
purchase_data_copy.drop_duplicates(subset="SN",keep="first", inplace=True)
purchase_data_copy["Age Group"] = pd.cut(purchase_data["Age"], bins, labels=labels)
age_groups = purchase_data_copy.groupby("Age Group")

In [25]:
player_count_by_age = age_groups["Age"].count()
player_count_by_age_per_100 = player_count_by_age/player_count*100

In [26]:
age_group_df = pd.DataFrame({"Player Count":player_count_by_age,"Percentage":player_count_by_age_per_100})
age_group_df

,Player Count,Percentage
Age Group,,
<10,17,2.951389
10-14,22,3.819444
15-19,107,18.576389
20-24,258,44.791667
25-29,77,13.368056
30-34,52,9.027778
35-39,31,5.381944
40+,12,2.083333


In [27]:
# Purchasing Analysis (Age)
purchase_data_copy = purchase_data.copy(deep="True")
purchase_data_copy["Age Group"] = pd.cut(purchase_data["Age"], bins, labels=labels)
age_groups = purchase_data_copy.groupby("Age Group")

In [29]:
purchase_count_by_age = age_groups["Purchase ID"].count()
avg_spent_by_age = age_groups["Price"].mean()
total_spent_by_age = age_groups["Price"].sum()
avg_spent_per_person = total_spent_by_age/player_count_by_age

In [30]:
age_summary = pd.DataFrame({"Purchase Count":purchase_count_by_age,
                               "Average Spent":avg_spent_by_age,
                               "Total Spent":total_spent_by_age,
                               "Average Spent Per Person":avg_spent_per_person})
age_summary

,Purchase Count,Average Spent,Total Spent,Average Spent Per Person
Age Group,,,,
<10,23,3.353478,77.13,4.537059
10-14,28,2.956429,82.78,3.762727
15-19,136,3.035956,412.89,3.858785
20-24,365,3.052219,1114.06,4.318062
25-29,101,2.900990,293.00,3.805195
30-34,73,2.931507,214.00,4.115385
35-39,41,3.601707,147.67,4.763548
40+,13,2.941538,38.24,3.186667


In [31]:
#Top Spenders
spenders = purchase_data.groupby("SN")

purchase_count_per_spender = spenders["SN"].count()
avg_value = spenders["Price"].mean()
total_value = spenders["Price"].sum()

spenders_summary = pd.DataFrame({"Purchase Count":purchase_count_per_spender,
                                "Average Purchase Price": avg_value,
                                "Total Purchase Value":total_value})


spenders_summary.sort_values(by="Total Purchase Value", ascending=False).head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,3.792000,18.96
Idastidru52,4,3.862500,15.45
Chamjask73,3,4.610000,13.83
Iral74,4,3.405000,13.62
Iskadarya95,3,4.366667,13.10


In [32]:
#Most Popular Items
items_df = purchase_data[["Item ID","Item Name", "Price"]].copy()
items_df = items_df.groupby(["Item ID","Item Name"])

num_items_sold = items_df["Item Name"].count()
max_price_sold_per_item = items_df["Price"].max()
total_sum_per_item = items_df["Price"].sum()

items_summary = pd.DataFrame({"Purchase Count":num_items_sold,
                            "Item Price":max_price_sold_per_item,
                             "Total Purchase Value":total_sum_per_item})

items_summary.sort_values(by="Purchase Count", ascending=False).head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,4.88,59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,4.23,50.76
145,Fiery Glass Crusader,9,4.58,41.22
132,Persuasion,9,3.33,28.99
108,"Extraction, Quickblade Of Trembling Hands",9,3.53,31.77


In [33]:
#Most Profitable Items
items_summary.sort_values(by="Total Purchase Value", ascending=False).head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,4.88,59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,4.23,50.76
82,Nirvana,9,4.90,44.10
145,Fiery Glass Crusader,9,4.58,41.22
103,Singed Scalpel,8,4.35,34.80
